In [1]:
!pip install dask-ml

In [2]:
!pip install aiohttp

In [3]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [4]:
import dask.dataframe as dd
from dask_ml.model_selection import train_test_split
from dask.distributed import Client, progress

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.metrics import roc_auc_score
import joblib

import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [5]:
client = Client(n_workers=4, threads_per_worker=2, memory_limit="2GB")
client

Client Scheduler: tcp://127.0.0.1:52947 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 8.00 GB


<IPython.core.display.Javascript object>

In [6]:
df = dd.read_csv(
    "https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/creditcard.csv",
    dtype={"Time": "float64"},
)

<IPython.core.display.Javascript object>

In [7]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


<IPython.core.display.Javascript object>

In [8]:
# creating x and y variables
X = df.drop(columns=["Class"])
y = df["Class"]

# splitting up the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=13
)

# Persisting the variables into RAM
X_train.persist()
X_test.persist()
y_train.persist()
y_test.persist()

Dask Series Structure:
npartitions=3
    int64
      ...
      ...
      ...
Name: Class, dtype: int64
Dask Name: split, 3 tasks

<IPython.core.display.Javascript object>

# Random Forest Model

In [9]:
rf = RandomForestClassifier()

with joblib.parallel_backend("dask"):
    rf.fit(X_train.compute(), y_train.compute())

preds_train = rf.predict(X_train.values.compute())
preds_test = rf.predict(X_test.values.compute())

print(
    "Logistic regression training score is: ",
    roc_auc_score(preds_train, y_train.values.compute()),
)
print(
    "Logistic regression test score is: ",
    roc_auc_score(preds_test, y_test.values.compute()),
)

distributed.client - ERROR - Error in callback <function DaskDistributedBackend.apply_async.<locals>.callback_wrapper at 0x000001BC801D4B88> of <Future: error, key: _parallel_build_trees-batch-9e44721f5cf443ef88df4061a828b5d5>:
Traceback (most recent call last):
  File "C:\Users\jlim7\anaconda3\lib\site-packages\distributed\client.py", line 285, in execute_callback
    fn(fut)
  File "C:\Users\jlim7\anaconda3\lib\site-packages\joblib\_dask.py", line 260, in callback_wrapper
    result = future.result()
  File "C:\Users\jlim7\anaconda3\lib\site-packages\distributed\client.py", line 220, in result
    raise exc.with_traceback(tb)
  File "C:\Users\jlim7\anaconda3\lib\site-packages\joblib\_dask.py", line 100, in __call__
    with parallel_backend('dask'):
  File "C:\Users\jlim7\anaconda3\lib\site-packages\joblib\parallel.py", line 186, in __init__
    backend = BACKENDS[backend](**backend_params)
  File "C:\Users\jlim7\anaconda3\lib\site-packages\joblib\_dask.py", line 137, in __init__
   

distributed.client - ERROR - Error in callback <function DaskDistributedBackend.apply_async.<locals>.callback_wrapper at 0x000001BC805C1438> of <Future: cancelled, key: _parallel_build_trees-batch-39ea5f2e5973427d96d76068eddcc8b3>:
Traceback (most recent call last):
  File "C:\Users\jlim7\anaconda3\lib\site-packages\distributed\client.py", line 285, in execute_callback
    fn(fut)
  File "C:\Users\jlim7\anaconda3\lib\site-packages\joblib\_dask.py", line 260, in callback_wrapper
    result = future.result()
  File "C:\Users\jlim7\anaconda3\lib\site-packages\distributed\client.py", line 222, in result
    raise result
concurrent.futures._base.CancelledError: _parallel_build_trees-batch-39ea5f2e5973427d96d76068eddcc8b3
distributed.client - ERROR - Error in callback <function DaskDistributedBackend.apply_async.<locals>.callback_wrapper at 0x000001BC805B0DC8> of <Future: cancelled, key: _parallel_build_trees-batch-8f6d5f3e5caa40f08d784cee29d3ed7d>:
Traceback (most recent call last):
  File 

RuntimeError: There is no current event loop in thread "Dask-Worker-Threads'-19764-3".

<IPython.core.display.Javascript object>